In [288]:
import pandas as pd
import sqlite3
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [289]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

In [290]:
import neattext as nt
import neattext.functions as nfx

In [291]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

sql = f"SELECT r.manual, (SELECT group_concat(rc.cat_id , ', ') " \
      f"from recipe_categories rc " \
      f"WHERE rc.recipe_id = r.id) AS categories from recipe r"

categories_sql = f"SELECT id from category c "

loaded_data = pd.read_sql(sql, conn)
loaded_categories = pd.read_sql(categories_sql, conn)
#pandas data frame
conn.close()

In [292]:
#print(loaded_categories.id)

ids_cat = []
for i in loaded_categories.id:
    ids_cat.append(str(i))

print(ids_cat)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

In [293]:
for i in range(len(ids_cat)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.categories[j]).split(', ')
        #print(spl)
        #print(int(str(ids_cat[i]) in spl))
        val.append(int(str(ids_cat[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
#print(val)

C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_3284\330240340.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_3284\330240340.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_3284\330240340.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

In [294]:
loaded_data

,manual,categories,1,2,3,4,5,6,7,8,...,594,595,596,597,598,599,600,601,602,603
0,Шоколад разломать на кусочки и вместе со сливо...,"1, 2, 3",1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Положите весь творог в кастрюльку и разомните ...,"4, 5, 6",0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Вскипятите воду в большой кастрюле и сварите п...,"7, 8, 9, 10",0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Разогреть духовку. Отделить белки от желтков. ...,"1, 5, 11",1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Взбить яйца с сахаром.\n\nПостепенно ввести му...,"1, 5, 12, 13, 14",1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,"Печенку посолить, поперчить, посыпать специями...","30, 5, 151",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6991,Крабовые палочки мелко нарезать или порубить в...,"17, 140, 106",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6992,"Чечевицу промыть, залить водой на 4 см выше ур...","37, 76",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6993,Нарезать филе средними полосками. Морковь наре...,"17, 67, 139",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
from nltk.corpus import stopwords

#nltk.download('stopwords')
stopwrds = set(stopwords.words('russian'))

In [296]:
import pymorphy2
import re

morph = pymorphy2.MorphAnalyzer()

In [297]:
def remove_stopwords(stroka, stops):
    stroka = re.sub(r'[^\w\s]','', stroka)
    stroka = stroka.lower().split()
    filt_text = ""
    for w in stroka:
        if w not in stops:
            filt_text += w + " "
    return filt_text

In [298]:
def text_normalization(text):
    filt_text = []

    for w in text:
        w_norm = morph.parse(w)[0]
        try:
            part_of_speech = w_norm.tag.POS
            w_norm = w_norm.normal_form+"_"+part_of_speech
        except TypeError as error:
            filt_text.append(w_norm.normal_form+"_UNKN")
        if w_norm not in filt_text:
            filt_text.append(w_norm)

    return filt_text

In [299]:
corpus = loaded_data['manual'].apply(lambda x: remove_stopwords(x, stopwrds))

In [300]:
print(corpus)

0       шоколад разломать кусочки вместе сливочным мас...
1       положите весь творог кастрюльку разомните вилк...
2       вскипятите воду большой кастрюле сварите пасту...
3       разогреть духовку отделить белки желтков белки...
4       взбить яйца сахаром постепенно ввести муку сол...
                              ...                        
6990    печенку посолить поперчить посыпать специями д...
6991    крабовые палочки мелко нарезать порубить бленд...
6992    чечевицу промыть залить водой 4 см выше уровня...
6993    нарезать филе средними полосками морковь нарез...
6994    очищенный вымытый картофель нарезать кубиками ...
Name: manual, Length: 6995, dtype: object


In [301]:
#corpus = loaded_data['manual'].apply(lambda x:nt.TextExtractor(x).extract_stopwords(stopwrds))

In [303]:
tfidf = TfidfVectorizer()

Xfeatures = tfidf.fit_transform(corpus).toarray()


MemoryError: Unable to allocate 1.30 GiB for an array with shape (6995, 24968) and data type float64

In [ ]:
Xfeatures

In [ ]:
y = loaded_data[ids_cat]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)

In [ ]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer().fit(x_train)
x_train = scaler.transform(x_train)

In [ ]:
binary_rel_clf = BinaryRelevance(MultinomialNB())

In [ ]:
binary_rel_clf.fit(x_train,y_train)

In [ ]:
br_prediction = binary_rel_clf.predict(x_test)

In [ ]:
br_prediction.toarray()

In [ ]:
accuracy_score(y_test,br_prediction)